### Propensity Parameter:  Generally-active vs available-and-participating population

We now add one parameter in our model, to capture the fact that propensity to participate in the survey may vary among workers. This propensity value captures both the inherent preference to participate in a survey, versus working on some other task, but also the general activity on the platform. For example, consider two workers that are equally likely to work on a survey; however, one of them works on the platform only one hour per week, while the other works for ten hours. Everything else being equal, the latter is ten times more likely to show up in our results.

For our model, we endow each user with a propensity parameter $a$, which defines the probability that a user will be **active and willing to participate in the survey** at any given time. The propensity parameter is a random variable with a pdf $p(a)$. Past literature suggests that the pdf $p(a)$ is a heavy tailed distribution, with many people participating only occasionally, and a small number of users are active substantially more. However, in our approach we do not make any strong assumptions about the specifics of the distribution.

#### Capture stage

If we have a population of $N$ users (not all of them active at the same time), each of them with a propensity $a_i$, then each user will be chosen by a probability proportional to $a_i$. 

$P( \mathit{capture} | a_i) = \frac{a_i}{\sum_j^N a_j} = \frac{a_i}{N \cdot E[a]}$.

So, the probability that we see worker $i$ in a sample of $n_1$ workers is the inverse of the probability of not-sampling the worker for $n_1$ samples:

$P( \mathit{sample} | a_i) = 1 - (1 - \frac{a_i}{\sum_j^N a_j})^{n_1} \approx \frac{n_1}{N} \cdot \frac{a_i}{E[a]} $

Given $a_i$, the probability of capturing the user in a second sample with $n_2$ workers is _conditionally_ independent, and therefore, the probability that a user will be captured in both the first and the second sample is:

$P( \mathit{capture},  \mathit{recapture} | a_i) = \frac{n_1}{N} \cdot \frac{a_i}{E[a]} \cdot \frac{n_2}{N} \cdot \frac{a_i}{E[a]} = \frac{n_1 \cdot n_2}{N^2 \cdot \left( E[a] \right)^2} \dot a_i^2 $

So, now we can form the expectation on the size of the intersection by integrating over all the possible workers:

$\sum_j^N P( \mathit{capture},  \mathit{recapture} | a_j) = 
N \cdot \int_0^1 P( \mathit{capture},  \mathit{recapture} | a)\cdot p(a) \, da =  
N \cdot \frac{n_1 \cdot n_2}{N^2 \cdot \left( E[a] \right)^2} \int_0^1 a^2 \cdot p(a) \, da = 
N \cdot \frac{n_1}{N} \cdot \frac{n_2}{N} \cdot \frac{E[a^2]}{E[a]^2}  = 
N \cdot \frac{n_1}{N} \cdot \frac{n_2}{N} \cdot \frac{Var(a) + E[a]^2}{E[a]^2} =  
N \cdot \frac{n_1}{N} \cdot \frac{n_2}{N} \cdot \left( 1 + \frac{Var(a)}{E[a]^2} \right) $

Notice that the vanilla capture-recapture considers the size of the intersection to be equal to $\frac{n_1}{N} \cdot \frac{n_2}{N}$. Notice that the factor $1 + \frac{Var(a)}{E[a]^2}$ becomes 1 when the variance $Var(a)$ of the propensity variable $a$ becomes zero. Therefore, when the propensity to be "captured" is the same across the population, or even when the variance is small compared to the mean propensity, the naive estimate is valid. However, when the ratio of standard deviation to the mean becomes big, then we start seeing systematic deviations. 

Intuition: When someone is sampled under our scheme, probabilistically that user is likely to have higher propensity to participate. Therefore, in the recapture period, we are more likely to sample the same people again. This leads to capture-recapture estimates for the population size $N$ that are systematically biased downwards.

* Example with propensity $a=0.5$ for everyone.
* Example with propensity uniform in the [0,1] interval. Average propensity is again $a=0.5$.
* Example with propensity following a Beta(a,b) with $a=b$ (i.e., $E[a]=0.5$), and $a \rightarrow 0$. Variance goes to $1/4$, and the naive estimate of the population is off by a factor of 2.
* Show example with heavy tailed distribution (e.g., $b=1$, $a=0.01$), showing that this can distort estimates significantly.

Generalizing the above, the factor $E[a^k] / E[a]^k$ factor that differentes the expected size of the intersection between $k$ sets when propensity is a random varialbe, and the naive estimate, which assumes that propensity is the same (or almost the same) across all members of the population.

In [1]:
# Create the overall population: Sample N users and give to each an activity/propensity value a

# Compute the average number of people that we expect to be active during that time. 
# That is the mean value of the distribution

# ===================

# Capture period

# Draw user from the population

# If user is active (random<a) proceed, else go to step 2

# Sample the user with probability 1/active users, and add id in the list

# Re-Capture period

# Repeat as above and compute estimate of population using the Lincoln estimator



In [48]:
# To instantiate using various probability functions take a look at 
# https://docs.scipy.org/doc/scipy/reference/stats.html
import numpy as np
from scipy.stats import beta

N = 100000
a = 0.08
b = 1
rvs = beta.rvs(a, b, loc= 0.02941, scale= 2.44, size=N)

# This transformation can help us transform any continuous RV 
# into a value ranging from 0 to 1
# p = [math.exp(rv)/(1+math.exp(rv)) for rv in rvs]
# The scipy.stats.rv_histogram should help us then transform the empirical
# values into an empirical distribution

mean, var, skew = beta.stats(a, b,  moments='mvs')
# For powerlaw on a bounded interval 0..1, it is a special case of Beta with b=1

print("Mean:", mean)
print("Var:", var)
print("Skew:", skew)
print("X1:", mean)
print("X2:", var+mean*mean)
print("X3:", skew*var*np.sqrt(var) + 3*mean*var + mean*mean*mean )
print("Common:", var+mean*mean)

factor = (var+mean*mean) / (mean*mean)
factor2 = (skew*var*np.sqrt(var) + 3*mean*var + mean*mean*mean) / ((var+mean*mean)*mean)
print("Factor:", factor )
print("Factor2:", factor2 )

Mean: 0.07407407407407407
Var: 0.03297457001160705
Skew: 3.0461675016268583
X1: 0.07407407407407407
X2: 0.0384615384615
X3: 0.025974025974
Common: 0.0384615384615
Factor: 7.00961538462
Factor2: 9.11688311688


In [49]:
sum(rvs)

21139.24579759804

In [50]:
%matplotlib inline
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 1)
ax.hist(rvs, normed=True, histtype='stepfilled', alpha=0.2, bins=100)
ax.set_yscale("log")

In [51]:
import numpy as np
# Given an instantiated list of propensity values 
# the mean of the distribution, and a sample target n, return the ids of the sampled entities
def sample(rvs, n):
    return sorted(np.random.choice(len(rvs), size = n, replace=True, p=rvs/sum(rvs)))


In [52]:
import statistics
from collections import Counter


tries = 1000
adj_estimates = []
naive_estimates = []
for _ in range(tries):
    n = 3000
      
    
    id1 = set(sample(rvs, n))
    id2 = set(sample(rvs, n))
    common = list(id1 & id2)
    
    #id3 = set(sample(rvs, n))
    #triad = list(id1 & id2 & id3)
    
     
    naive_estimate = len(id1) * len(id2) / len(common)
    #naive_estimate = len(common) * len(id3) / len(triad)
    naive_estimates.append(naive_estimate)
    
    adj_estimate = naive_estimate * factor
    #adj_estimate = naive_estimate * factor2
    adj_estimates.append(adj_estimate)                       
    
print("Naive Median:", statistics.median(naive_estimates))
print("Naive Mean:", statistics.mean(naive_estimates))
                                 
print("Adj Median:", statistics.median(adj_estimates))
print("Adj Mean:", statistics.mean(adj_estimates))

Naive Median: 19708.800039246467
Naive Mean: 19732.589146271228
Adj Median: 138151.107967
Adj Mean: 138317.860458


(array([  8.33870519e-06,   0.00000000e+00,   0.00000000e+00,
          1.66774104e-05,   8.33870519e-06,   8.33870519e-06,
          1.66774104e-05,   4.16935260e-05,   5.00322311e-05,
          6.67096415e-05,   1.33419283e-04,   1.33419283e-04,
          1.41757988e-04,   1.41757988e-04,   2.41822451e-04,
          2.58499861e-04,   3.91919144e-04,   2.83515976e-04,
          3.83580439e-04,   3.50225618e-04,   4.41951375e-04,
          4.75306196e-04,   4.25273965e-04,   5.42015837e-04,
          4.91983606e-04,   3.66903028e-04,   4.16935260e-04,
          3.25209502e-04,   3.33548208e-04,   3.08532092e-04,
          2.75177271e-04,   2.50161156e-04,   1.66774104e-04,
          1.83451514e-04,   1.33419283e-04,   9.17257571e-05,
          1.16741873e-04,   5.83709363e-05,   5.83709363e-05,
          5.00322311e-05,   5.83709363e-05,   4.16935260e-05,
          1.66774104e-05,   1.66774104e-05,   0.00000000e+00,
          8.33870519e-06,   0.00000000e+00,   0.00000000e+00,
        

In [53]:
fig, ax = plt.subplots(1, 1)
ax.hist(adj_estimates, normed=True, histtype='stepfilled', alpha=0.2, bins=50)
ax.hist(naive_estimates, normed=True, histtype='stepfilled', alpha=0.2, bins=50)

In [54]:
from collections import Counter
sampled_ids = []
samples = 27
n = 3000
for _ in range(samples):
    s = set(sample(rvs, n))
    sampled_ids.extend(s)
z = Counter(Counter(sampled_ids).values())
z

Counter({1: 13831,
         2: 4284,
         3: 2714,
         4: 2061,
         5: 1565,
         6: 1204,
         7: 942,
         8: 646,
         9: 418,
         10: 269,
         11: 127,
         12: 70,
         13: 29,
         14: 7,
         15: 4,
         16: 4})